In [1]:
path = 'C:/Users/ISHMO_CT/Downloads/my-projects/nueral_env/Loan-Approval-Prediction/playground-series-s4e10/train.csv'

In [2]:
import pandas as pd
df = pd.read_csv(path)

In [3]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [5]:
# covert the categorical varaiables into numerical by label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['person_home_ownership'] = le.fit_transform(df['person_home_ownership'])
df['loan_intent'] = le.fit_transform(df['loan_intent'])
df['loan_grade'] = le.fit_transform(df['loan_grade'])
df['cb_person_default_on_file'] = le.fit_transform(df['cb_person_default_on_file'])

In [6]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,1,22,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,2,29,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,3,30,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,4,22,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0


In [7]:
# split the data into target and features 
target = 'loan_status'
y = df[target]
X = df.drop(columns=target)

In [8]:
# split the train set into train and validation set 
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
# scale the data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.fit_transform(X_val)

In [14]:
X_train.shape

(52780, 12)

In [28]:
# build the model
# z=Wx+b 
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(12,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()


c:\Users\ISHMO_CT\.virtualenvs\nueral_env-PneuDF7g\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# compile the model 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# train the model with the data 
import os
import datetime
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", f"Normaliztion-{now}")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=tensorboard)

Epoch 1/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9458 - loss: 0.1812 - val_accuracy: 0.9420 - val_loss: 0.1825
Epoch 2/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9461 - loss: 0.1759 - val_accuracy: 0.9458 - val_loss: 0.1784
Epoch 3/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9447 - loss: 0.1791 - val_accuracy: 0.9436 - val_loss: 0.1827
Epoch 4/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9466 - loss: 0.1745 - val_accuracy: 0.9422 - val_loss: 0.1796
Epoch 5/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9468 - loss: 0.1736 - val_accuracy: 0.9412 - val_loss: 0.1791
Epoch 6/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9459 - loss: 0.1766 - val_accuracy: 0.9437 - val_loss: 0.1775
Epoch 7/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9456 - loss: 0.1764 - val_accuracy: 0.9442 - val_loss: 0.1748
Epoch 8/10
1650/1650 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9462 - loss: 0.1756 - 

In [24]:
import tensorboard
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17336), started 0:00:20 ago. (Use '!kill 17336' to kill it.)

In [30]:
# First create a function for the model 
def create_model(optimizer='adam', epochs=10, batch_size=32, units=64):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(12,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [34]:
# wrap the tensorflow keras model for GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
model = KerasClassifier(build_fn=create_model, units=64)

In [35]:

# Tune the model 
from sklearn.model_selection import GridSearchCV
# create the parameters 
param_grid = {
    'optimizer':['adam', 'rmsprop', 'sgd'],
    'epochs': [5, 10, 15],
    'batch_size': [16, 32, 64],
    'units': [32, 64, 128]
}

In [ ]:
# perform the gid search 
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train, validation_data=(X_val, y_val))

In [39]:
# evaluate the best model on valdstion data
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
best_model =grid_result.best_estimator_
y_pred = best_model.predict(X_val)
y_pred_class = [1 if x > 0.5 else 0 for x in y_pred]

print("Validation set accurancy:", accuracy_score(y_val, y_pred_class))
print("Classification Report: \n", classification_report(y_val, y_pred_class))
print("Confusion Matrix: \n", confusion_matrix(y_val, y_pred_class))

367/367 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Validation set accurancy: 0.9437340153452686
Classification Report: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      5041
           1       0.90      0.68      0.77       824

    accuracy                           0.94      5865
   macro avg       0.92      0.83      0.87      5865
weighted avg       0.94      0.94      0.94      5865

Confusion Matrix: 
 [[4978   63]
 [ 267  557]]


In [42]:
# test set prediction 
test_set = 'C:/Users/ISHMO_CT/Downloads/my-projects/nueral_env/Loan-Approval-Prediction/playground-series-s4e10/test.csv'
df_test = pd.read_csv(test_set)
df_test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [44]:
# transfor the test_set 
df_test['person_home_ownership'] = le.fit_transform(df_test['person_home_ownership'])
df_test['loan_intent'] = le.fit_transform(df_test['loan_intent'])
df_test['loan_grade'] = le.fit_transform(df_test['loan_grade'])
df_test['cb_person_default_on_file'] = le.fit_transform(df_test['cb_person_default_on_file'])

In [45]:
df_test

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,3,3.0,2,5,25000,15.76,0.36,0,2
1,58646,26,96000,0,6.0,4,2,10000,12.68,0.10,1,4
2,58647,26,30000,3,5.0,5,4,4000,17.19,0.13,1,2
3,58648,33,50000,3,4.0,0,0,7000,8.90,0.14,0,7
4,58649,26,102000,0,8.0,2,3,15000,16.32,0.15,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,22,31200,0,2.0,0,1,3000,10.37,0.10,0,4
39094,97739,22,48000,0,6.0,1,0,7000,6.03,0.15,0,3
39095,97740,51,60000,0,0.0,4,0,15000,7.51,0.25,0,25
39096,97741,22,36000,0,4.0,4,3,14000,15.62,0.39,1,4


In [46]:
y_pred_test = best_model.predict(df_test)

2444/2444 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [47]:
df_pred = pd.DataFrame({
    'id': df_test['id'],
    'loan_status': y_pred_test
})

In [48]:
df_pred.to_csv("neural_submission.csv", index=False)